# noaa-weather-hourly
This script cleans and formats a manually downloaded "National Oceanic and Atmospheric Administration" (NOAA) "LCD CSV" weather file.  This is the development version of the script, not the publishable version.

 __Originated From:__
https://github.com/emskiphoto/Process-historical-NOAA-LCD-weather<BR>
    
Copyright January 1, 2025
https://github.com/emskiphoto/noaa-weather-hourly
Matt Chmielewski

### Load Python packages

In [1]:
# direct the jupyter notebook development script to add package path to PYTHONPATH
# to allow for loading of source modules.
import sys
sys.path.append('../noaa-weather-hourly')

In [34]:
# import modules specific to this package 
from config import *
from utils import *

In [3]:
import pandas as pd
import csv
import pathlib
import re
# turn off Jedi autocomplete (that was causing more problems than benefits post Win10 update 3-13-2020)
%config Completer.use_jedi = False 

### Parameters - Store in config.py

In [4]:
is_development = True

## Locations

In [5]:
dir_cwd = pathlib.Path.cwd()
dir_cwd

WindowsPath('C:/Users/user/OneDrive/python_envs/noaa-weather-hourly-cli/dev')

In [6]:
# using 'data' within 'dev' folder - potentially different from 'data' in package folder
# dir_data should be dir_cwd in production script
if is_development:
    dir_data = dir_cwd / 'data'
else:
    dir_data = dir_cwd
assert dir_data.is_dir()

#### Are there any .CSV files of any naming format?
If not, stop the script, there is nothing to do without the local .csv's.

In [8]:
# pretty name of directory
dir_data_posix = dir_data.as_posix()
# list of all .csv files in dir_data
dir_data_csv_files = sorted([f_.name for f_ in dir_data.glob('*.csv') if f_.is_file()])

In [9]:
# if some .csv files are present, continue.  Otherwise halt process and inform user.
try:
    assert len(dir_data_csv_files) >= 1
except:
    message_ = message_no_csv_files_found.format(dir_data_posix = dir_data_posix)
    print(message_)
#     assert
    raise
dir_data_csv_files_str = ', '.join(dir_data_csv_files)

### Locate LCD .CSV file(s) 'files_lcd_input'
This script is intended to be executed from a terminal command line.  The LCD input file(s) are expected to be saved in the same directory that the command line is executed in.  The file name(s) are expected to match the pattern associated with multiple LCD file versions in 'patterns_lcd_input_files' (two versions currently).  However, if a file(s) with this pattern is not identifed, do NOT attempt to use any non-matching .CSV file in the same directory.  Inform user that no matching file was found and no files will be opened or created.

The benefits of this approach are:
1. code will not mistakenly use non-LCD files
2. User can be sloppy (or organized) with their LCD file storage.  New source files and output files can simply be accumulated in the same folder with no data loss.
3. Simple command line requires no mandatory input, only optional frequency and parameter setting inputs.

In [10]:
patterns_lcd_input_files

['^[0-9]{5,10}.csv', '^LCD_(.*)_[0-9]{4}.csv']

In [11]:
# which version of LCD files are avaialble and which are the most recent?
# 1. find all files that match v1 or v2 naming
# 2. find the most recent file
# 3. Determine if most recent file is v1 or v2 format 'lcd_version'
# 4. see if there is more than one file with the same station ID
# 5. create list 'files_lcd' with one or more lcd files of same station id 

In [12]:
# 1. find all files that match v1 or v2 naming and sort by last modified date descending
# create 'version_files' dictionary with LCD version number as key and list of matching 
# files as values
version_files = {v_ : find_files_re_pattern_sorted_last_modified(dir_data, pattern_) for
                 v_, pattern_ in version_pattern_lcd_input.items()}
version_files

{1: [WindowsPath('C:/Users/user/OneDrive/python_envs/noaa-weather-hourly-cli/dev/data/3876540.csv'),
  WindowsPath('C:/Users/user/OneDrive/python_envs/noaa-weather-hourly-cli/dev/data/3875753.csv')],
 2: [WindowsPath('C:/Users/user/OneDrive/python_envs/noaa-weather-hourly-cli/dev/data/LCD_USW00014939_2023.csv'),
  WindowsPath('C:/Users/user/OneDrive/python_envs/noaa-weather-hourly-cli/dev/data/LCD_USW00014939_2022.csv'),
  WindowsPath('C:/Users/user/OneDrive/python_envs/noaa-weather-hourly-cli/dev/data/LCD_USW00014939_2021.csv'),
  WindowsPath('C:/Users/user/OneDrive/python_envs/noaa-weather-hourly-cli/dev/data/LCD_USW00014939_2020.csv'),
  WindowsPath('C:/Users/user/OneDrive/python_envs/noaa-weather-hourly-cli/dev/data/LCD_USW00094846_2023.csv'),
  WindowsPath('C:/Users/user/OneDrive/python_envs/noaa-weather-hourly-cli/dev/data/LCD_USW00094846_2022.csv'),
  WindowsPath('C:/Users/user/OneDrive/python_envs/noaa-weather-hourly-cli/dev/data/LCD_USW00094846_2021.csv'),
  WindowsPath('C:/Us

In [13]:
files_pattern_match = [x for xs in version_files.values() for x in xs]

In [14]:
# what if no files were found?  
# return message and halt process
# which files matched lcd patterns, regardless of version or date?
# files_pattern_match = [x for xs in version_files.values() for x in xs]
try:
    assert len(files_pattern_match) >= 1
except:
    message_ = message_no_lcd_files_found.format(dir_data_posix = dir_data_posix,
                                 patterns_lcd_examples_str = patterns_lcd_examples_str,
                                dir_data_csv_files_str = dir_data_csv_files_str)
    print(message_)
#     assert
    raise

In [15]:
# find most recently modified file by lcd version
version_file_last_modified = {version_ : files_[0] for version_, files_ in version_files.items()}
version_file_last_modified

{1: WindowsPath('C:/Users/user/OneDrive/python_envs/noaa-weather-hourly-cli/dev/data/3876540.csv'),
 2: WindowsPath('C:/Users/user/OneDrive/python_envs/noaa-weather-hourly-cli/dev/data/LCD_USW00014939_2023.csv')}

In [16]:
# 2. find the most recent file
file_last_modified = sorted([(f, f.stat().st_mtime) for
                  f in version_file_last_modified.values()],
           key=lambda x: x[1], reverse=True)[0][0]
# 3. Determine if most recent file is v1 or v2 format 'lcd_version'
# versions start with '1' so need to add 1 to zero-indexed list
lcd_version = list(version_file_last_modified.values())\
                            .index(file_last_modified) + 1
# file_last_modified, lcd_version

In [17]:
# make sure we have the right version
assert file_last_modified in version_files[lcd_version]

### Group LCD .CSV input file(s) as 'files_lcd_input'

In [18]:
# 4. see if there is more than one file that has the same station ID
# as found in 'file_last_modified'
# This requires extraction of a unique identifier in LCD file name that is common to
# other LCD files for same location (but probably different dates).  

# Note that only the LCD v2 files need to be grouped.  LCD v1 files are 
# delivered with multi-year date ranges (if requested) while LCD v2
# files are for discrete calendar years (or less), for example 'LCD_USW00014939_2020.csv'.  

# Grouping LCD v1 files could be implemented, but this would require cooperation
# from the user in terms of renaming the LCD v1 files in a specific format.
# LCD v1 files are delivered with the same name ('3876540.csv') regardless
# of date range of data in file.    

In [19]:
# empty list to hold LCD input files
files_lcd_input = []
# different treatment for v2 LCD files
if lcd_version == 2:
# extract id_file_lcd2 as the blob of characters between first and second '_'
# reference 'LCD_USW00014939_2023.csv'
    id_file_lcd2 = file_last_modified.name.split('_')[1]
#     which files contain id for the current lcd_version?
    files_ = [file_ for file_ in version_files[lcd_version] if id_file_lcd2 in file_.name]
    files_lcd_input.extend(files_)
#     print('v2')
else:
    files_lcd_input.extend(file_last_modified)
files_lcd_input

[WindowsPath('C:/Users/user/OneDrive/python_envs/noaa-weather-hourly-cli/dev/data/LCD_USW00014939_2023.csv'),
 WindowsPath('C:/Users/user/OneDrive/python_envs/noaa-weather-hourly-cli/dev/data/LCD_USW00014939_2022.csv'),
 WindowsPath('C:/Users/user/OneDrive/python_envs/noaa-weather-hourly-cli/dev/data/LCD_USW00014939_2021.csv'),
 WindowsPath('C:/Users/user/OneDrive/python_envs/noaa-weather-hourly-cli/dev/data/LCD_USW00014939_2020.csv')]

### Keep only files that have one or more columns from cols_noaa_processed

#### Which columns are present by file in files_lcd_input?

In [49]:
# read only headers of each file in files_lcd_input
files_columns = {}
for file_ in files_lcd_input:
    try:
#         this is 30x faster than pd.read_csv(file_, index_col=0, nrows=0).columns.tolist()
        with open(file_, 'r') as infile:
            reader = csv.DictReader(infile)
            fieldnames = reader.fieldnames
        files_columns[file_] = sorted(fieldnames)
    except:
        continue

### Create files_usecols containing valid files and columns to be used

#### Is 'DATE' available for every file in files_lcd_input?

In [50]:
# keep only files that have a 'DATE' column - otherwise where is this data supposed to go?
files_usecols = {file_ : cols_ for file_, cols_ in files_columns.items() if 'DATE' in cols_}
files_usecols

{WindowsPath('C:/Users/user/OneDrive/python_envs/noaa-weather-hourly-cli/dev/data/LCD_USW00014939_2023.csv'): ['BackupDirection',
  'BackupDistance',
  'BackupDistanceUnit',
  'BackupElements',
  'BackupElevation',
  'BackupEquipment',
  'BackupLatitude',
  'BackupLongitude',
  'BackupName',
  'CoolingDegreeDaysSeasonToDate',
  'DATE',
  'DailyAverageDewPointTemperature',
  'DailyAverageDryBulbTemperature',
  'DailyAverageRelativeHumidity',
  'DailyAverageSeaLevelPressure',
  'DailyAverageStationPressure',
  'DailyAverageWetBulbTemperature',
  'DailyAverageWindSpeed',
  'DailyCoolingDegreeDays',
  'DailyDepartureFromNormalAverageTemperature',
  'DailyHeatingDegreeDays',
  'DailyMaximumDryBulbTemperature',
  'DailyMinimumDryBulbTemperature',
  'DailyPeakWindDirection',
  'DailyPeakWindSpeed',
  'DailyPrecipitation',
  'DailySnowDepth',
  'DailySnowfall',
  'DailySustainedWindDirection',
  'DailySustainedWindSpeed',
  'DailyWeather',
  'ELEVATION',
  'HeatingDegreeDaysSeasonToDate',
  'H

#### Keep only files that have at least one cols_data column

In [51]:
files_usecols = {file_ : cols_ for file_, cols_ in files_usecols.items()
                 if len(set(cols_).intersection(set(cols_data))) >=1}
files_usecols

{WindowsPath('C:/Users/user/OneDrive/python_envs/noaa-weather-hourly-cli/dev/data/LCD_USW00014939_2023.csv'): ['BackupDirection',
  'BackupDistance',
  'BackupDistanceUnit',
  'BackupElements',
  'BackupElevation',
  'BackupEquipment',
  'BackupLatitude',
  'BackupLongitude',
  'BackupName',
  'CoolingDegreeDaysSeasonToDate',
  'DATE',
  'DailyAverageDewPointTemperature',
  'DailyAverageDryBulbTemperature',
  'DailyAverageRelativeHumidity',
  'DailyAverageSeaLevelPressure',
  'DailyAverageStationPressure',
  'DailyAverageWetBulbTemperature',
  'DailyAverageWindSpeed',
  'DailyCoolingDegreeDays',
  'DailyDepartureFromNormalAverageTemperature',
  'DailyHeatingDegreeDays',
  'DailyMaximumDryBulbTemperature',
  'DailyMinimumDryBulbTemperature',
  'DailyPeakWindDirection',
  'DailyPeakWindSpeed',
  'DailyPrecipitation',
  'DailySnowDepth',
  'DailySnowfall',
  'DailySustainedWindDirection',
  'DailySustainedWindSpeed',
  'DailyWeather',
  'ELEVATION',
  'HeatingDegreeDaysSeasonToDate',
  'H

In [53]:
# reduce files_usecols to only columns used in this process
files_usecols = {file_ : sorted(set(cols_noaa_processed).intersection(set(cols_))) for
                 file_, cols_ in files_usecols.items()}
files_usecols

### Open and Concatenate all files in files_usecols

In [55]:
files_usecols

{WindowsPath('C:/Users/user/OneDrive/python_envs/noaa-weather-hourly-cli/dev/data/LCD_USW00014939_2023.csv'): ['DATE',
  'HourlyAltimeterSetting',
  'HourlyDewPointTemperature',
  'HourlyDryBulbTemperature',
  'HourlyPrecipitation',
  'HourlyPressureChange',
  'HourlyRelativeHumidity',
  'HourlyStationPressure',
  'HourlyVisibility',
  'HourlyWetBulbTemperature',
  'HourlyWindDirection',
  'HourlyWindGustSpeed',
  'HourlyWindSpeed',
  'STATION',
  'Sunrise',
  'Sunset'],
 WindowsPath('C:/Users/user/OneDrive/python_envs/noaa-weather-hourly-cli/dev/data/LCD_USW00014939_2022.csv'): ['DATE',
  'HourlyAltimeterSetting',
  'HourlyDewPointTemperature',
  'HourlyDryBulbTemperature',
  'HourlyPrecipitation',
  'HourlyPressureChange',
  'HourlyRelativeHumidity',
  'HourlyStationPressure',
  'HourlyVisibility',
  'HourlyWetBulbTemperature',
  'HourlyWindDirection',
  'HourlyWindGustSpeed',
  'HourlyWindSpeed',
  'STATION',
  'Sunrise',
  'Sunset'],
 WindowsPath('C:/Users/user/OneDrive/python_envs

In [58]:
df = pd.concat((pd.read_csv(f_, usecols=cols_, parse_dates=['DATE'],
                            index_col='DATE', low_memory=False) for
                f_, cols_ in files_usecols.items()), axis=0).sort_index()
df

,STATION,HourlyAltimeterSetting,HourlyDewPointTemperature,HourlyDryBulbTemperature,HourlyPrecipitation,HourlyPressureChange,HourlyRelativeHumidity,HourlyStationPressure,HourlyVisibility,HourlyWetBulbTemperature,HourlyWindDirection,HourlyWindGustSpeed,HourlyWindSpeed,Sunrise,Sunset
DATE,,,,,,,,,,,,,,,
2020-01-01 00:00:00,USW00014939,NaN,-6.1,-2.8,NaN,-1.9,78.0,967.9,NaN,-3.9,220.0,NaN,2.6,NaN,NaN
2020-01-01 00:00:00,USW00014939,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,751.0,1709.0
2020-01-01 00:00:00,USW00014939,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2020-01-01 00:54:00,USW00014939,1009.8,-7.8,-5.0,0.0,NaN,81.0,967.2,16.093,-5.9,170.0,NaN,2.6,NaN,NaN
2020-01-01 01:54:00,USW00014939,1008.1,-7.8,-4.4,0.0,NaN,77.0,965.5,16.093,-5.5,170.0,NaN,2.6,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-12-31 20:54:00,USW00014939,1027.8,-6.1,-3.3,NaN,1.2,81.0,NaN,16.093,NaN,300,NaN,1.5,NaN,NaN
2023-12-31 21:00:00,USW00014939,NaN,-6.1,-3.3,NaN,1.2,81.0,984.3,16.0,-4.3,300,NaN,1.5,NaN,NaN
2023-12-31 21:54:00,USW00014939,1027.8,-7.8,-5.6,NaN,NaN,85.0,NaN,16.093,NaN,000,NaN,0.0,NaN,NaN


In [59]:
# df = df[cols_use].copy()
df.shape

(50283, 15)

In [60]:
df.head()

,STATION,HourlyAltimeterSetting,HourlyDewPointTemperature,HourlyDryBulbTemperature,HourlyPrecipitation,HourlyPressureChange,HourlyRelativeHumidity,HourlyStationPressure,HourlyVisibility,HourlyWetBulbTemperature,HourlyWindDirection,HourlyWindGustSpeed,HourlyWindSpeed,Sunrise,Sunset
DATE,,,,,,,,,,,,,,,
2020-01-01 00:00:00,USW00014939,NaN,-6.1,-2.8,NaN,-1.9,78.0,967.9,NaN,-3.9,220.0,NaN,2.6,NaN,NaN
2020-01-01 00:00:00,USW00014939,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,751.0,1709.0
2020-01-01 00:00:00,USW00014939,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2020-01-01 00:54:00,USW00014939,1009.8,-7.8,-5.0,0.0,NaN,81.0,967.2,16.093,-5.9,170.0,NaN,2.6,NaN,NaN
2020-01-01 01:54:00,USW00014939,1008.1,-7.8,-4.4,0.0,NaN,77.0,965.5,16.093,-5.5,170.0,NaN,2.6,NaN,NaN


### Identify and Display Weather Station Information

In [72]:
# v1 & v2
# identify WBAN station
station_lcd = str(df['STATION'].value_counts().index[0])
station_wban = station_lcd[6:]
station_lcd, station_wban

('USW00014939', '14939')

#### Open 'isd-history.csv' containing Station details
This is supplemental, non-essential information.   It is important that any errors with this step do not negatively impact the core process.

In [73]:
isd_history_available = False
file_isd_history = find_latest_file(dir_data, pattern_isd_history_file)
if file_isd_history.is_file():
    isd_history_available = True

In [74]:
df_isd_history = pd.read_csv(file_isd_history, index_col='WBAN',
                             dtype={'WBAN': object}).sort_values(by=['USAF', 'BEGIN'],
                                                             ascending=[True, False])
df_isd_history.sample(5).sort_index()

,USAF,STATION NAME,CTRY,ST,CALL,LAT,LON,ELEV(M),BEGIN,END
WBAN,,,,,,,,,,
00124,949999,WAIKERIE (E&WS),AS,NaN,NaN,-34.180,+140.000,+0030.0,1957-01-01,1972-04-28
00141,949999,MOUNT SURPRISE,AS,NaN,NaN,-18.150,+144.320,+0453.0,1956-12-31,1978-09-10
00209,A05735,BOWIE MUNICIPAL AIRPORT,US,TX,K0F2,+33.600,-097.783,+0336.2,2012-01-01,2024-12-15
54770,726163,JAFFREY MINI-SLVR RNCH APT,US,NH,KAFN,+42.807,-072.001,+0308.4,2006-01-01,2024-12-15
93044,723626,CURRITUCK CO,US,NC,KONX,+36.400,-076.017,+0005.0,1973-01-01,1973-01-08


In [97]:
df_isd_history.loc[station_wban].reset_index()

,WBAN,USAF,STATION NAME,CTRY,ST,CALL,LAT,LON,ELEV(M),BEGIN,END
0,14939,725510,LINCOLN MUNICIPAL AIRPORT,US,NE,KLNK,+40.848,-096.765,+0356.7,1942-10-01,2024-12-16
1,14939,999999,LINCOLN AIR BASE,US,NE,KLNK,+40.851,-096.748,+0362.4,1948-01-01,1972-12-31


In [98]:
# df_isd_history.loc[int(station_wban)]
# using station_wban, gather station details from isd history for the record with the most
# recent 'END' date
station_details = dict(df_isd_history.loc[station_wban].reset_index()\
                       .sort_values('END', ascending=False).iloc[0])
station_details

{'WBAN': '14939',
 'USAF': '725510',
 'STATION NAME': 'LINCOLN MUNICIPAL AIRPORT',
 'CTRY': 'US',
 'ST': 'NE',
 'CALL': 'KLNK',
 'LAT': '+40.848',
 'LON': '-096.765',
 'ELEV(M)': '+0356.7',
 'BEGIN': '1942-10-01',
 'END': '2024-12-16'}

In [78]:
# remove 'STATION', 'REPORT_TYPE', 'SOURCE' columns - not needed anymore
df.drop(columns=['STATION', 'REPORT_TYPE', 'SOURCE'], inplace=True, errors='ignore')

In [79]:
#     create timestamps for weather data time series
start_dt = df.index[0]
end_dt = df.index[-1]
start_str = start_dt.strftime('%Y-%m-%d')
end_str = end_dt.strftime('%Y-%m-%d')
start_str, end_str

('2020-01-01', '2023-12-31')

In [80]:
df.dtypes

HourlyAltimeterSetting       float64
HourlyDewPointTemperature    float64
HourlyDryBulbTemperature     float64
HourlyPrecipitation           object
HourlyPressureChange         float64
HourlyRelativeHumidity       float64
HourlyStationPressure        float64
HourlyVisibility              object
HourlyWetBulbTemperature     float64
HourlyWindDirection           object
HourlyWindGustSpeed          float64
HourlyWindSpeed              float64
Sunrise                      float64
Sunset                       float64
dtype: object

In [81]:
cols_dtypes = {'HourlyAltimeterSetting': 'float64',
   'HourlyDewPointTemperature': 'float64',
    'HourlyDryBulbTemperature': 'float64',
    'HourlyPrecipitation': 'float64',
    'HourlyPressureChange': 'float64',
    'HourlyRelativeHumidity': 'Int64',
    'HourlyStationPressure': 'float64',
    'HourlyVisibility': 'float64',
    'HourlyWetBulbTemperature': 'float64',
    'HourlyWindDirection': 'Int64',
    'HourlyWindGustSpeed': 'float64',
    'HourlyWindSpeed': 'float64',
    'Sunrise': 'datetime64[ns]',
    'Sunset': 'datetime64[ns]'}
# keep only items with corresponding column in df
cols_dtypes = {k : v for k, v in cols_dtypes.items() if k in df.columns}

In [82]:
for col_, type_ in cols_dtypes.items():
    if type_ == 'float64':
        df[col_] = pd.to_numeric(df[col_], errors='coerce')
        try:
            df[col_] = df[col_].astype(type_)
        except:
            pass
    elif type_ == 'Int64':
        df[col_] = pd.to_numeric(df[col_], errors='coerce')
        try:
            df[col_] = df[col_].astype(type_)
        except:
            pass

In [83]:
df.dtypes

HourlyAltimeterSetting       float64
HourlyDewPointTemperature    float64
HourlyDryBulbTemperature     float64
HourlyPrecipitation          float64
HourlyPressureChange         float64
HourlyRelativeHumidity         Int64
HourlyStationPressure        float64
HourlyVisibility             float64
HourlyWetBulbTemperature     float64
HourlyWindDirection            Int64
HourlyWindGustSpeed          float64
HourlyWindSpeed              float64
Sunrise                      float64
Sunset                       float64
dtype: object

In [84]:
# forward fill 'Sunrise' & 'Sunset' data
cols_sunrise_sunset = df.columns.intersection(['Sunrise', 'Sunset']).tolist()
for col_ in cols_sunrise_sunset:
    print(col_)
    time = pd.to_datetime(pd.to_numeric(df[col_].ffill()\
                             .bfill()), format = '%H%M')
    YMDHM = pd.DataFrame({'Year': df.index.year,
                         'Month' : df.index.month,
                        'Day' : df.index.day,
                        'Hour' : time.dt.hour,
                         'Minute' : time.dt.minute})

    df[col_] = pd.to_datetime(YMDHM)
    del time, YMDHM
df[cols_sunrise_sunset].sample(5)

Sunrise
Sunset


,Sunrise,Sunset
DATE,,
2020-07-07 16:54:00,2020-07-07 05:02:00,2020-07-07 20:01:00
2022-12-21 23:41:00,2022-12-21 07:47:00,2022-12-21 17:02:00
2022-06-08 00:12:00,2022-06-08 04:56:00,2022-06-08 19:56:00
2022-01-28 21:54:00,2022-01-28 07:42:00,2022-01-28 17:38:00
2021-04-06 19:54:00,2021-04-06 06:04:00,2021-04-06 18:56:00


# STOPPED HERE - 01/01/25
Review criteria for null timestamps - v2 files may need different logic

In [85]:
# are there timestamps that have a high count of null values?
n_records_hourly_approx = int(df.shape[0]/(24))
n_max_null = pct_null_timestamp_max * n_records_hourly_approx

In [86]:
temp = df.loc[:, df.columns.difference(cols_sunrise_sunset)]
df_nan_ts = temp.groupby(temp.index.time).apply(lambda x: x.isna().sum()\
                            .gt(n_max_null)).all(axis=1)
times_nan = df_nan_ts.loc[df_nan_ts].index.tolist()
del temp
del df_nan_ts

In [95]:
pct_null_timestamp_max

0.5

In [94]:
n_max_null

1047.5

In [87]:
times_nan

[datetime.time(0, 0)]

In [88]:
# remove records for timestamps with a high percentage of Null values.
# note that the '23:59:00' timestamp is suspect and appears to only be a placeholder
# for posting sunrise/sunset times.  Important that this step be done after
# forward filling sunrise/sunset values.
filter_nan_times = pd.Series(df.index.time).isin(times_nan).values
# df = df.loc[~filter_nan_times]
# df.info()

In [93]:
test = df.between_time('23:01:00', '00:59:00').select_dtypes(include=[int, float])
test.groupby(test.index.time).sum()

,HourlyAltimeterSetting,HourlyDewPointTemperature,HourlyDryBulbTemperature,HourlyPrecipitation,HourlyPressureChange,HourlyRelativeHumidity,HourlyStationPressure,HourlyVisibility,HourlyWetBulbTemperature,HourlyWindDirection,HourlyWindGustSpeed,HourlyWindSpeed
00:00:00,2019.6,5909.6,13355.7,1.7,429.2,102080,1390264.6,3130.854,9881.6,249289,0.0,5134.7
00:01:00,9151.4,73.1,97.4,0.0,0.0,760,8765.3,100.179,84.5,1820,18.0,42.8
00:02:00,8103.7,29.9,42.3,0.7,0.0,720,7761.7,61.556,36.8,1380,26.8,41.7
00:03:00,8129.1,52.2,76.7,0.5,0.0,648,7786.0,113.456,66.6,2100,14.4,43.8
00:04:00,6056.2,53.9,65.0,1.4,0.0,531,5800.4,73.223,59.2,500,11.3,32.5
...,...,...,...,...,...,...,...,...,...,...,...,...
23:54:00,1482354.9,6054.4,13670.9,100.1,438.3,104213,1411051.9,22073.600,10033.1,252780,1994.1,5220.0
23:56:00,2024.7,32.8,36.6,0.2,0.0,178,1939.2,18.507,34.2,390,13.9,10.8
23:57:00,1010.8,-10.0,-8.9,0.0,0.0,92,968.2,0.000,-9.2,360,0.0,1.5
23:58:00,4041.0,41.1,45.0,2.9,0.0,374,3870.2,17.702,43.0,460,12.9,20.1


In [89]:
df.loc[filter_nan_times]

,HourlyAltimeterSetting,HourlyDewPointTemperature,HourlyDryBulbTemperature,HourlyPrecipitation,HourlyPressureChange,HourlyRelativeHumidity,HourlyStationPressure,HourlyVisibility,HourlyWetBulbTemperature,HourlyWindDirection,HourlyWindGustSpeed,HourlyWindSpeed,Sunrise,Sunset
DATE,,,,,,,,,,,,,,
2020-01-01,NaN,-6.1,-2.8,NaN,-1.9,78,967.9,NaN,-3.9,220,NaN,2.6,2020-01-01 07:51:00,2020-01-01 17:09:00
2020-01-01,NaN,NaN,NaN,NaN,NaN,<NA>,NaN,NaN,NaN,<NA>,NaN,NaN,2020-01-01 07:51:00,2020-01-01 17:09:00
2020-01-01,NaN,NaN,NaN,NaN,NaN,<NA>,NaN,NaN,NaN,<NA>,NaN,NaN,2020-01-01 07:51:00,2020-01-01 17:09:00
2020-01-02,NaN,-2.8,0.6,NaN,-0.4,78,953.5,NaN,-0.7,<NA>,NaN,0.0,2020-01-02 07:51:00,2020-01-02 17:09:00
2020-01-02,NaN,NaN,NaN,NaN,NaN,<NA>,NaN,NaN,NaN,<NA>,NaN,NaN,2020-01-02 07:52:00,2020-01-02 17:10:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-12-29,NaN,-1.1,1.7,NaN,-1.5,82,975.8,16.0,0.6,330,NaN,5.7,2023-12-29 07:51:00,2023-12-29 17:07:00
2023-12-30,NaN,NaN,NaN,NaN,NaN,<NA>,NaN,NaN,NaN,<NA>,NaN,NaN,2023-12-30 07:51:00,2023-12-30 17:07:00
2023-12-30,NaN,-4.4,-1.7,NaN,-0.3,82,974.4,16.0,-2.7,0,NaN,0.0,2023-12-30 07:51:00,2023-12-30 17:07:00


In [ ]:
#     Check what percentage of data has null data and print to screen
df_pct_null_data = pd.DataFrame({'Percent N/A': df.isnull().sum().divide(len(df)).round(3)})
df_pct_null_data_formatted = df_pct_null_data['Percent N/A'].apply(lambda n: '{:,.1%}'.format(n))
# remove 'Hourly' prefix for display only
col_rename_remove_hourly = {col_ : col_.replace('Hourly', '') for
                            col_ in df_pct_null_data_formatted.index}

In [ ]:
message_pct_null_data = f"""Percent Missing Values by Column for LCD source file '{file_lcd_input.name}' for USAF station {station_usaf} at '{station_details['STATION NAME']}' from {start_str} to {end_str}."""
print(message_pct_null_data)
display(df_pct_null_data_formatted.rename(index=col_rename_remove_hourly))

In [ ]:
# create dictionary 'col_null_policy' to manually guide the resolution of null values by column
# in general, data that is expected to have a non-zero/non-null reading at all times
# will have nulls filled through interpolation.  Data that normally has 0 or null readings
# like 'HourlyPrecipitation' will have null values filled with 0 in the interest of having
# a 100% clean dataset to facilitate manual analysis.
col_null_policy = {'HourlyAltimeterSetting': pd.Series.interpolate,
 'HourlyDewPointTemperature': pd.Series.interpolate,
 'HourlyDryBulbTemperature': pd.Series.interpolate,
 'HourlyPrecipitation': lambda x: x.fillna(0),
 'HourlyPressureChange': pd.Series.interpolate,
 'HourlyRelativeHumidity': pd.Series.interpolate,
 'HourlyStationPressure': pd.Series.interpolate,
 'HourlyVisibility': pd.Series.interpolate,
 'HourlyWetBulbTemperature': pd.Series.interpolate,
 'HourlyWindDirection': pd.Series.interpolate,
 'HourlyWindGustSpeed': lambda x: x,
 'HourlyWindSpeed': pd.Series.interpolate,
 'Sunrise': pd.Series.ffill,
 'Sunset': pd.Series.bfill}
# keep only items with corresponding column in df
col_null_policy = {k : v for k, v in col_null_policy.items() if k in df.columns}
# assert len(df.columns.intersection(col_null_policy.keys())) == df.shape[1]

In [ ]:
df.dtypes

In [ ]:
# df['HourlyWindSpeed'].astype(float).interpolate()

In [ ]:
df['HourlyAltimeterSetting']

In [ ]:
col = 'HourlyAltimeterSetting'
# df[col].resample('H').agg(col_null_policy[col])

pd.to_numeric(df[col], errors='coerce').dropna().resample(freqstr).mean()

In [ ]:
# create dictionary 'col_resample_policy' to manually guide resampling method by column

col_resample_policy = {'HourlyAltimeterSetting': 'mean',
 'HourlyDewPointTemperature': 'mean',
 'HourlyDryBulbTemperature': 'mean',
 'HourlyPrecipitation': 'mean',
 'HourlyPressureChange': 'mean',
 'HourlyRelativeHumidity': 'mean',
 'HourlyStationPressure': 'mean',
 'HourlyVisibility': 'mean',
 'HourlyWetBulbTemperature': 'mean',
 'HourlyWindDirection': 'mean',
 'HourlyWindGustSpeed': 'mean',
 'HourlyWindSpeed': 'mean',
 'Sunrise': pd.Series.mode,
 'Sunset': pd.Series.mode}
# keep only items with corresponding column in df
col_resample_policy = {k : v for k, v in col_resample_policy.items() if k in df.columns}
# assert len(df.columns.intersection(col_resample_policy.keys())) == df.shape[1]

In [ ]:
df['Sunrise'].dtype.name

# STOPPED HERE

In [ ]:
pd.Series.resample()

In [ ]:
%%time
dfs = {}
for col_ in df.columns:
    print(col_)
#     if df[col_].dtype.name == 'datetime64[ns]':
#         df[col_].resample(freqstr).agg(col_resample_policy[col_])
#     else:
#     df[col_].agg(col_null_policy[col_])
#         pd.to_numeric(df[col], errors='coerce').dropna().resample(freqstr).agg(col_resample_policy[col_])
    dfs[col_] = df[col_].dropna().resample(freqstr).agg(col_resample_policy[col_])

In [ ]:
df_out = pd.concat(dfs, axis=1)
df_out

In [ ]:
df_out.info()

In [ ]:
# df_out[cols_sunrise_sunset].diff(axis=1).iloc[:,-1].dt.total_seconds().div(3600).plot()

In [ ]:
# df.agg(col_null_policy).astype(cols_dtypes)
# df.agg(col_null_policy).isna().sum()

In [ ]:
# transform remaining null values per the 'col_null_policy'
# df = df.agg(col_null_policy).astype(cols_dtypes)
# display(pd.DataFrame({'Percent N/A': df.isnull().sum().divide(len(df))\
#                       .round(3)}).apply(lambda n: '{:,.1%}'.format(n)))

In [ ]:
#     Check what percentage of data has null data and print to screen
df_pct_null_data_post = pd.DataFrame({'Percent N/A': df_out.isnull().sum().divide(len(df_out)).round(3)})
df_pct_null_data_post_formatted = df_pct_null_data_post['Percent N/A'].apply(lambda n: '{:,.1%}'.format(n))
display(df_pct_null_data_post_formatted)

In [ ]:
df_pct_null_data_formatted

In [ ]:
for col_ in df.columns:
    print(col_)
    df[col_].resample(freqstr)

In [ ]:
df.dtypes

In [ ]:
pd.to_numeric(df['HourlyVisibility'])

In [ ]:
pd.to_datetime(df['Sunrise'])

In [ ]:
df.resample(freqstr).mean()

In [ ]:
#     Resample data to produce evenly spaced interval data
# df = df.resample(freqstr).median()
df = df.resample(freqstr).mean()

### Quality Control

#### No duplicate records

#### Complete and consistent timestamps

#### correct dtypes

#### Nan values only in columns permitted to have NaN

## Create Additional Columns (not implemented)

In [ ]:
# #     assign seasons to df in new 'season' column
#     df = weather_assign_season(df)
# #     assign weekend or weekday tag to each day
#     df['weekday'] = 'weekday'
#     df.loc[~df.index.weekday.isin(list(range(0,5))),'weekday'] = 'weekend'
#     df['weekday'] = df['weekday'].astype('category')   # keep memory size to a minimum
# #     Add True/False Peak Coincident column
#     df = is_month_and_hour_in_list(df, new_column='Peak Coincident', months=[6,7,8,9],hours=[14,15,16,17])
# #     Add holiday True/False:
#     years = list(df.index.year.unique())
#     try:
#         holiday_dates = generate_list_of_holidays(years=years)
#         holiday_YMD = []
#         for d in holiday_dates:
#             holiday_YMD.append(datetime.datetime(d.year, d.month, d.day).date())
#         df['holiday'] = [dat in holiday_YMD for dat in df.index.date]
# #         alternatively:
# # holiday_dates = generate_list_of_holidays(years=df.index.year.unique().to_list())
# # df['Date_copy_delete'] = df.index.date.copy()
# # df['holiday'] = df.Date_copy_delete.apply(lambda date: date in holiday_dates)
# # df.drop(columns=['Date_copy_delete'], inplace=True)
#     except:
#         print("Holidays were not tagged.  Veritfy it 'Holidays' python package is installed")
# #     Add temperature bins
#     temp_bins = list(range(-30,110,5))
#     df['DryBulbTemperature_bin'] = pd.cut(df['HourlyDryBulbTemperature'],bins=temp_bins)

### Export df to Disk

#### Rename Columns - remove 'hourly' from names

#### Name export file 

In [ ]:
# city = input("Input city or project name:  ")
# export_name = "{}_".format(city) + start_str + "_to_" + end_str + f"_{df.index.freqstr}.csv"
# export_name

In [ ]:
file_output_format.format(STATION_NAME = 'billy',
                         )

In [ ]:
file_out_name = f"{station_details['']city) + start_str + "_to_" + end_str + f"_{df.index.freqstr}.csv"
file_out = dir_cwd / file_out_name
file_out_name

In [ ]:
# os.path.join(os.path.dirname(file),export_name)

In [ ]:
[col.replace('Hourly','') for col in df.columns]

In [ ]:
df.columns

In [ ]:
df.to_csv(os.path.join(os.path.dirname(file),export_name))
# os.path.join(os.path.dirname(directory),f'All_M_V_data {todays_date}.csv')

# END